# Домашнее задание

## Задание № 1
Напишите функцию, которая возвращает название валюты (поле ‘Name’) с максимальным значением курса с помощью сервиса www.cbr-xml-daily.ru...ly_json.js


In [8]:
import requests

In [113]:
def currency_max_value():
    # отправляем запрос на сайт:
    get_r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
    # обрабатываем ответ:
    request_result = get_r.json()
    request_result 
    max_currency_value = float("-inf")
    for item, value in request_result.items():
        # item - содержит названия пунктов первого уровня
        # value - содержит значения для пунктов первого уровня
        # далее действия выполняются для 'Valute':
        if item == 'Valute':
            # print(item)
            for currency, info in value.items():
                # currency - сокращённое наименование валюты
                # info - атрибуты валюты
                currency_name = info['Name'] # имя валюты
                currency_value = info['Value'] # валюта в руб.
                
                # находим максимальную валюту:
                if currency_value > max_currency_value:
                    max_currency_value = currency_value

    print(max_currency_value, currency_name)

In [114]:
a = currency_max_value()
a

101.9217 Японских иен


## Задание 2
Добавьте в класс Rate параметр diff (со значениями True или False), который в случае значения True в методах курсов валют (eur, usd итд) будет возвращать не курс валюты, а изменение по сравнению в прошлым значением. Считайте, self.diff будет принимать значение True только при возврате значения курса. При отображении всей информации о валюте он не используется.

In [36]:
class Rate:
    # создаём метод "конструктор класса" или "конструктор объектов класса".
    # конструктор класса - автоматически вызывается при создании объекта
    # он содержит основные атрибуты / переменные / парамерты:
    def __init__(
            self,                   # указывает на экземпляр самого объекта
            format_ = 'value',      # value - возвращает только значение, # full - возвращает всю инф. по валюте
            diff = 'False'          # False - возвращает текущий курс, True - возвращает разницу между периодами
            ):
        self.format = format_
        self.diff_attribute = diff

    # создаём метод, который подключается к источнику данных и возвращает инф. по всем валютам:
    def exchange_rates(self):
        """
        Возвращает ответ сервиса с информацией о валютах в виде:
        
        {
            'AMD': {
                'CharCode': 'AMD',
                'ID': 'R01060',
                'Name': 'Армянских драмов',
                'Nominal': 100,
                'NumCode': '051',
                'Previous': 14.103,
                'Value': 14.0879
                },
            ...
        }
        """
        self.r = requests.get('https://www.cbr-xml-daily.ru/daily_json.js')
        return self.r.json()['Valute']
    
    # создаём метод, в котором регулируется формат вывода данных по любой валюте:
    def make_format(
            self,       
            currency    # валюта, которую указал пользователь
            ):
        """
        Возвращает информацию о валюте currency в двух вариантах:
        - полная информация о валюте при self.format = 'full':
        Rate('full').make_format('EUR')
        {
            'CharCode': 'EUR',
            'ID': 'R01239',
            'Name': 'Евро',
            'Nominal': 1,
            'NumCode': '978',
            'Previous': 79.6765,
            'Value': 79.4966
        }
        
        Rate('value').make_format('EUR')
        79.4966
        """
        
        # переменная с результатом ответа сервиса:
        response = self.exchange_rates()
        # переменная с полным описанием валюты:
        currency_full_value = response[currency]
        # переменная со значением валюты:
        currency_value = response[currency]['Value']
        # переменная со значением валюты предыдущего периода:
        previous_period_currency_value = response[currency]['Previous']
        # переменная с разницей текущего курса и предыдущего:
        exchange_rate_diff = currency_value - previous_period_currency_value
        
        # если код валюты имеется в запросе
        if currency in response:
            
            # если пользователь захотел увидеть всю информацию
            if self.format == 'full':
                # венуть полное описание запрошенной валюты
                return currency_full_value
                
            # если пользователь хочет увидеть только значение курса
            if self.format == 'value':
                # и разницу между текущим и прошлым периодом
                if self.diff_attribute == 'True':
                    # вернуть сравнение курса по валюте
                   return exchange_rate_diff
                # наверное пользователь хочет просто курс валюты:
                else: return currency_value
        
        return 'Error'

    # ниже создаются методы, которые настроены на возврат данных по конкретной валюте:
    def eur(self):
        """Возвращает курс евро на сегодня в формате self.format"""
        return self.make_format('EUR')
    
    def usd(self, diff = 'False'):
        """Возвращает курс доллара на сегодня в формате self.format"""
        return self.make_format('USD')
    
    def brl(self, diff = 'False'):
        """Возвращает курс бразильского реала на сегодня в формате self.format"""
        return self.make_format('BRL')

In [118]:
# подключаемся к классу -> создаём объект:
r = Rate(format_='value', diff = 'True') 

In [119]:
# смотрим изменение курса относительно прошлого периода:
r.make_format(currency='USD')

0.1595000000000084

In [120]:
r.usd()

0.1595000000000084

## Задание 3
Напишите класс Designer, который учитывает количество международных премий для дизайнеров (из презентации: “Повышение на 1 грейд за каждые 7 баллов. Получение международной премии – это +2 балла”). Считайте, что при выходе на работу сотрудник уже имеет две премии и их количество не меняется со стажем (конечно если хотите это можно вручную менять).

Класс Designer пишется по аналогии с классом Developer из материалов занятия.

Разбор условий:
 - На работу берут только тех дизайнеров, у которых есть 2 международные премии = 4 балла;
 - Получение международной премии = +2 балла;
 - Прохождение сертификации = +1 балл;
 - Каждый седьмой балл = повышение грейда.

In [107]:
# Родитеслький класс "Сотрудник"
class Employee:
    def __init__(
            self,           # указывает на экземпляр самого объекта
            name,           # имя сотрудника
            seniority,      # стаж в баллах
            ):
        
        self.name = name
        self.seniority = seniority
        self.grade = 1      # значение грейда по умолчанию 
    
    def grade_up(self):
        """Повышает уровень сотрудника"""
        self.grade += 1
    
    def publish_grade(self):
        """Публикация результатов аккредитации сотрудников"""
        print(self.name, self.grade)

    # метод для просмотра кол-ва баллов сотрудника:
    def publish_seniority(self):
        """Публикация количества баллов сотрудника"""
        print(self.name, self.seniority)

    # метод для просмотра греда и кол-ва баллов сотрудника:
    def publish_employee_info(self):
        """Публикация грейда и количества баллов сотрудника"""
        print("Имя: " + str(self.name) + "; Баллы: " + str(self.seniority) + "; Грейд: " + str(self.grade))

    def check_if_it_is_time_for_upgrade(self):
        pass

In [108]:
# Для каждого отдела заводится свой класс
class Designer(Employee):
    # создаём метод, который будет наследовать класс:
    def __init__ (
            self,
            name,
            seniotiry = 4   # 4 балла на старте, т.к. устраивают только тех, у кого 2 премии
            ):
        # вызываем метод __init__ класса Employee:
        super().__init__(name, seniotiry)

    # метод, для повышения балла за пройденную аккредитацию:
    def employee_has_been_accredited(self):
        """
        Начисление 1-го балла сотруднику за пройденную аккредитацию.
        Если количество баллов будет достаточным - грейд будет повышен.
        """
        # повышение балла
        self.seniority += 1

        # повышение грейда, если достаточно баллов:
        if self.seniority % 7 == 0:
            self.grade_up()
        # публикация результатов
        return self.publish_employee_info()  

    # метод, для повышения балла за международную премию:
    def employee_has_got_international_award(self):
        """
        Начисление 2-х баллов сотруднику за за международную премию.
        Если количество баллов будет достаточным - грейд будет повышен.
        """
        # повышение балла
        self.seniority += 2

        # повышение грейда, если достаточно баллов:
        if self.seniority % 7 == 0:
            self.grade_up()
        # публикация результатов
        return self.publish_employee_info()   


In [109]:
Alex = Designer('Александр Пушкин')

In [110]:
Alex.publish_employee_info()

Имя: Александр Пушкин; Баллы: 4; Грейд: 1


In [111]:
# проверка. словно сотрудник 3 раза прошёл аккредитацию:
for i in range(3):
    Alex.employee_has_been_accredited()

Имя: Александр Пушкин; Баллы: 5; Грейд: 1
Имя: Александр Пушкин; Баллы: 6; Грейд: 1
Имя: Александр Пушкин; Баллы: 7; Грейд: 2


In [112]:
# проверка. словно сотрудник 2 раза получил медународную премию:
for i in range(3):
    Alex.employee_has_got_international_award()

Имя: Александр Пушкин; Баллы: 9; Грейд: 2
Имя: Александр Пушкин; Баллы: 11; Грейд: 2
Имя: Александр Пушкин; Баллы: 13; Грейд: 2
